
**Comentario del Revisor**

Hola!

Soy Juan Manuel Romero, pero siéntete libre de llamarme Juanma. Soy code reviewer en Tripleten y hoy estaré revisando tu entrega.

Para simular la dinámica de un ambiente de trabajo, si veo algún error, en primer instancia solo los señalaré, dándote la oportunidad de encontrarlos y corregirlos por tu cuenta. En un trabajo real, el líder de tu equipo hará una dinámica similar. En caso de que no puedas resolver la tarea, te daré una información más precisa en la próxima revisión. 

Solo un aviso rápido: cuando estés revisando el proyecto, por favor deja mis comentarios originales tal como están. De esta manera, podemos seguir fácilmente el progreso y asegurarnos de que no se nos pase nada por alto. Y, si realizas algún cambio basado en mis comentarios, sería genial si pudieras resaltar esas actualizaciones para que se destaquen.

Puedes encontrar mis comentarios en cajas verdes, amarillas o rojas como estas:

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor</b> <a class="tocSkip"></a>

Éxito. Todo se ha hecho correctamente.

</div>


<div class="alert alert-block alert-warning"> 
<b>Comentario del Revisor</b> <a class="tocSkip"></a>

Observaciones. Algunas recomendaciones.

</div> 


<div class="alert alert-block alert-danger">
<b>Comentario del Revisor</b> <a class="tocSkip"></a>

Requiere corrección. El bloque requiere algunas correcciones. El trabajo no puede ser aceptado con los comentarios en rojo.

</div>

Puedes responderme usando esto:

<div class="alert alert-block alert-info"> <b>Respuesta del estudiante.</b> <a class="tocSkip"></a> </div>

# Proyecto_13

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

### a) Cargar y analizar los datos

In [25]:
#Se importan librerias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import lightgbm as lgb

In [2]:
#Se cargan los datos
data = pd.read_csv('car_data.csv')


In [3]:
#Se imprimen el DataSet
print(data.head(5))


        DateCrawled  Price VehicleType  RegistrationYear Gearbox  Power  \
0  24/03/2016 11:52    480         NaN              1993  manual      0   
1  24/03/2016 10:58  18300       coupe              2011  manual    190   
2  14/03/2016 12:52   9800         suv              2004    auto    163   
3  17/03/2016 16:54   1500       small              2001  manual     75   
4  31/03/2016 17:25   3600       small              2008  manual     69   

   Model  Mileage  RegistrationMonth  FuelType       Brand NotRepaired  \
0   golf   150000                  0    petrol  volkswagen         NaN   
1    NaN   125000                  5  gasoline        audi         yes   
2  grand   125000                  8  gasoline        jeep         NaN   
3   golf   150000                  6    petrol  volkswagen          no   
4  fabia    90000                  7  gasoline       skoda          no   

        DateCreated  NumberOfPictures  PostalCode          LastSeen  
0  24/03/2016 00:00               

In [4]:
#Se obtiene la informacion del DataSet
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

### b) Limpieza de datos

In [5]:
#Filtramos valores irreales o atipicos
data = data[(data['Price'] > 0) &(data['Price'] < 100000)]
data = data[(data['RegistrationYear'] > 1900) & (data['RegistrationYear'] < 2024)]

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Muy bien! Has estudiado los rangos de los valores y eliminado observaciones fuera de lo esperado. Esto evita que los valores atípicos o erróneos influyan negativamente en el análisis.

</div>

In [6]:
#Trabajamos con los valores Null
print(data.isnull().sum())

DateCrawled              0
Price                    0
VehicleType          33611
RegistrationYear         0
Gearbox              17243
Power                    0
Model                17468
Mileage                  0
RegistrationMonth        0
FuelType             29356
Brand                    0
NotRepaired          65748
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64


In [7]:
data['VehicleType'].fillna('Uknown', inplace=True)
data['Gearbox'].fillna("Uknown", inplace=True)
data['Model'].fillna('Uknown', inplace=True)
data['FuelType'].fillna('Uknown', inplace=True)
data['NotRepaired'].fillna('Uknown', inplace=True)

print(data.isnull().sum())

/var/folders/tl/svswzrd161q7xy22grsvz20h0000gn/T/ipykernel_2712/2220305061.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['VehicleType'].fillna('Uknown', inplace=True)
/var/folders/tl/svswzrd161q7xy22grsvz20h0000gn/T/ipykernel_2712/2220305061.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64


<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Excelente! Procesaste correctamente los valores incorrectos y faltantes. El tratamiento de estos datos es esencial para evitar sesgos y errores en el entrenamiento del modelo.

</div>

In [8]:
#Se eliminan columnas irrelevantes

data = data.drop(columns = ['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'])

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Muy bien! Eliminaste las características innecesarias, como las fechas y el índice. Esto ayuda a reducir el ruido en el análisis y mejora la eficiencia del modelo al centrarse en variables relevantes. ¡Gran decisión!

</div>

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 343458 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              343458 non-null  int64 
 1   VehicleType        343458 non-null  object
 2   RegistrationYear   343458 non-null  int64 
 3   Gearbox            343458 non-null  object
 4   Power              343458 non-null  int64 
 5   Model              343458 non-null  object
 6   Mileage            343458 non-null  int64 
 7   RegistrationMonth  343458 non-null  int64 
 8   FuelType           343458 non-null  object
 9   Brand              343458 non-null  object
 10  NotRepaired        343458 non-null  object
dtypes: int64(5), object(6)
memory usage: 31.4+ MB


## Entrenamiento del modelo 

### a) Pre-procesamiento

In [10]:
#Codificacion de variables caategoricas

data = pd.get_dummies(data, drop_first=True) 

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Buen trabajo codificando las características categóricas! Esto convierte variables cualitativas en números que el modelo puede procesar.

</div>

In [11]:
#Se divide los datos en festures y target

features = data.drop(columns = 'Price')
target = data['Price']

In [12]:
#Dividir los datos en train, valid y test

X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size = 0.25, random_state=12345)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=12345)

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Genial! Dividiste los datos correctamente. 
</div>

In [13]:
#Se escala las caracteristicas

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.fit_transform(X_valid)
X_test_scaled = scaler.fit_transform(X_test)

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Veo que has analizado varios modelos, excelente! Comparar varios algoritmos es una estrategia clave, ya que permite seleccionar el más adecuado para tu conjunto de datos.

</div>

### b) Entrenamiento de modelo Regresion Lineal

In [14]:
model_lr = LinearRegression()
model_lr.fit(X_train_scaled, y_train)
predictions_lr = model_lr.predict(X_valid_scaled)
rmse_lr = np.sqrt(mean_squared_error(y_valid, predictions_lr))
print("RMSE - Linear Regresion", rmse_lr)

RMSE - Linear Regresion 2873.59418148578


##### La regresion lineal tiene el mayor RMSE entre todos los modelos comparados, lo que sugiere que no captura bien las relaciones complejas en los datos. La regresion lineal es un modelo simple y no esta diseñado para manejar interacciones no lineales o complejas

### c) Arbol de decision

In [15]:
model_dt = DecisionTreeRegressor(max_depth=100, random_state=12345)
model_dt.fit(X_train_scaled, y_train)
predictions_dt = model_dt.predict(X_valid_scaled)
rmse_dt = np.sqrt(mean_squared_error(y_valid, predictions_dt))
print("RMSE - Decision Tree", rmse_dt)

RMSE - Decision Tree 2117.4927398463665


##### El arbol de decision mejora considerablemente el RMSE en comparacion con la regresion lineal. Esto demuestra que es capaz de capturar mejor las relaciones no lineales entre las caracteristicas y el precio del vehiculo. Tiene un rendimiento limitado, posiblemente debido a un sobreajuste.

### d) Bosque Aleatorio

In [16]:
model_rf = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=12345)
model_rf.fit(X_train_scaled, y_train)
predictions_rf = model_rf.predict(X_valid_scaled)
rmse_rf = np.sqrt(mean_squared_error(y_valid, predictions_rf))
print("RMSE - Random Forest", rmse_rf)

RMSE - Random Forest 1737.134944154855


##### El bosque aleatorio reduce aun mas el RMSE, combinando varios arboles de decision mejora significativamente la capacidad predictiva. Este modelo es muy robusto y mejora la variabilidad en los datos al reducir el sobreajuste.

### e) LightGBM 

In [17]:
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

In [18]:
params = {
    'objective' : 'regression',
    'metric' : 'rmse',
    'boosting_type' : 'gbdt',
    'learning_rate' : 0.1,
    'max_depth' : 10,
    'num_leaves' : 31  
}


<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Muy buen ajuste de hiperparámetros. Esto es crucial para optimizar el rendimiento del modelo, ya que los hiperparámetros controlan el comportamiento interno del algoritmo.

</div>

In [24]:
model_lgb = lgb.train(params, train_data, valid_sets=[valid_data])
predictions_lgb = model_lgb.predict(X_valid)
rmse_lgb = np.sqrt(mean_squared_error(y_valid, predictions_lgb))
print("RMSE - LightGBM", rmse_lgb)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015451 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 947
[LightGBM] [Info] Number of data points in the train set: 257593, number of used features: 296
[LightGBM] [Info] Start training from score 4551.645002
RMSE - LightGBM 1767.9433718613452


##### LightGBM tiene un RMSE ligeramente mayor que el bosque aleatorio, pero es un modelo bastante competitivo. Su mensake nos indica que se eligio correctamente un enfoque de procesamiento eficiente y se utilizo 296 caracteristicas relevantes para el entrenamiento. Es un modelo mucho mas rapido en entrenar que el bosque aleatorio, el cual es se demoro mucho en su entrenamiento, lo que nos indique que en momentos donde el tiempo es crucial nos puede ayudar mucho.

### f) CatBoost 

In [26]:
model_cb = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=10, loss_function='RMSE', verbose=0, random_state=12345)
model_cb.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=10)
predictions_cb = model_cb.predict(X_valid)
rmse_cb = np.sqrt(mean_squared_error(y_valid, predictions_cb))
print("RMSE - CatBoost", rmse_cb)

RMSE - CatBoost 1584.5195444137019


##### El modelo de CatBoost es el mejor ya que logra el RMSE mas preciso, su capacidad para manejar caracteristicas categoricas directamente lo combientes en el mejor modelo.

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Excelente trabajo entrenando y validando los diferentes modelos! Además, dejas comentarios claros sobre los resultados en cada caso.

</div>

### g) Conclusiones

##### El modelo con el mejor desempeño es el CatBoost

##### El modelo mas eficiente en cuestion de tiempo es el LightGBM

##### La regresion lineal no es un modelo preciso pero sirve como una base de referencia para comparaciones.


## Análisis del modelo

### a) Regresion Lineal

In [31]:
final_predictions_lr = model_lr.predict(X_test)
final_rmse_lr = np.sqrt(mean_squared_error(y_test, final_predictions_lr))
print("Final RMSE - Lineal Regression", final_rmse_lr)

Final RMSE - Lineal Regression 179008629.60121644


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


##### El RMSE extremadamente ato suguiere que la regresion lineal no puede capturar adecuadamente las relaciones entre las caracteristicas y el precio del coche.

### b) Arbol de decision

In [35]:
final_predictions_dt = model_dt.predict(X_test)
final_rmse_dt = np.sqrt(mean_squared_error(y_test, final_predictions_dt))
print('Final RMSE - Decision Tree', final_rmse_dt)


Final RMSE - Decision Tree 4383.007839250821


/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


##### El arbol de decision es propenso al sobreajuste de los datos de entrenamiento, por lo que no generaliza de correcta forma los datos nuevos.

### c) Bosque aleatorio

In [34]:
final_predictions_rf = model_rf.predict(X_test)
final_rmse_rf = np.sqrt(mean_squared_error(y_test, final_predictions_rf))
print('Final RMSE - Random Forest', final_rmse_rf)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


Final RMSE - Random Forest 4423.219081723359


##### El aumento del RMSE con respecto a los conjuntos de entrenamiento sugiere que el modelo esta sobreajustado, es decir aprende patrones especificos de los datos de entrenamiento pero no generaliza bien a datos desconocidos.

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Correcto, en caso de seguir con estos modelos habría que volver al entrenamiento para hacer los ajustes necesarios.

</div>

### d) LightGBM

In [39]:
final_predictions_lgb = model_lgb.predict(X_test)
final_rmse_lgb = np.sqrt(mean_squared_error(y_test, final_predictions_lgb))
print( "Final RMSE - LightGBM", final_rmse_lgb)

Final RMSE - LightGBM 1760.3276402095687


<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Fantástico! Lograste un valor de RMSE menor a 2500, lo que indica que tu modelo tiene una buena capacidad predictiva. RMSE es una métrica clave para evaluar el error medio de las predicciones en problemas de regresión.

</div>

##### LightGBM es muy superior a los modelos anteriores, su precision es muy cercana a CatBoost y con una gran velocidad, un gran modelo con excelentes resultados

### e) CatBoost

In [40]:
final_predictions_cb = model_cb.predict(X_test)
final_rmse_cb = np.sqrt(mean_squared_error(y_test, final_predictions_cb))
print("Final RMSE - CatBoost:", final_rmse_cb)

Final RMSE - CatBoost: 1591.7417539566513


##### CatBoost es el modelo mas preciso de todos, su velocidad no es superior a LightGBM pero su precision lo vale.

<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Perfecto! Comparaste el rendimiento final del modelo con el conjunto de prueba, lo que es correcto. Esta etapa es clave para evaluar la capacidad del modelo de generalizarse a nuevos datos.

</div>

### f) Conclusiones

##### CatBoost resulto ser el modelo mas preciso 

##### LightGBM tiene un rendimiento cercano a CatBoost pero una mayor rapidez de entrenamiento

<div class="alert alert-block alert-warning"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Aunque haces un análisis del tiempo, no veo que quede reflejado en el trabajo. Entiendo que miraste el tiempo en cada celda de código para esto, sin embargo, lo correcto es utilizar una librería como _time_ para calcular el tiempo de forma precisa e imprimir en pantalla los tiempos para que quede correctamente registrado. 

</div>

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos


<div class="alert alert-block alert-success"> 
<b>Comentario del Revisor #1</b> <a class="tocSkip"></a>

Felicidades Gabriel! Tu trabajo es excelente.

Has completado todos los items necesarios para aprobar la entrega.

</div>